In [1]:
using Pkg, Revise
Pkg.activate("/home/jovyan/elementary-linear-algebra/GenLinAlgProblems")
using GenLinAlgProblems, LinearAlgebra, Latexify, LaTeXStrings, Random, StatsBase, IJulia

Random.seed!( 12345 );

  Activating project at `~/elementary-linear-algebra/GenLinAlgProblems`


<div style="float:center;width:100%;text-align: center;">
    <strong style="height:60px;color:darkred;font-size:40px;">Solving the Lights Out Puzzle</strong><br>
</div>

In [2]:
# show the puzzle
url = "https://people.math.carleton.ca/~kcheung/math/apps/lights_out.html"
iframe_html = """
<iframe src="$url" width="500px" height="610px"></iframe>
<div style="float:left;width:40%;padding-right:1cm;padding-top:1cm;">

<strong style="font-size:30px;">Let's model the Lights Out puzzle using Linear Algebra!</strong>
</div>
"""
display("text/html", iframe_html)

Let's model the Lights Out puzzle using Linear Algebra!

# 1. Modeling the Puzzle

## 1.1 Use Modulo Two Arithmetic

The game consists of a number of cells arranged in a rectangular array that have two states. Let's label them **Red** and **Black**.

Clicking on a cell toggles the stat of the cell and its direct neighbors (the cells with which it shares a common edge).<br>
The object of the game is to turn all cells **Black**.

<strong style="color:blue;font-size:15px;"> Encoding and Toggling the State of a Cell</strong>

Toggling the cell state can be modeled using the addition operation from **modulo two arithmetic:**<br>
Encode **Black** as 0 and **Red** as 1, and observe that modulo two arithmetic correctly toggles the state.

Let $a$ be the current state of a cell, and add $0$ to leave the state as is, or $1$ to toggle the state:
* $a = 0:\qquad a+0 = 0,\;\; a+1 = 1\qquad\qquad (mod 2)$
* $a = 1:\qquad a+0 = 1,\;\; a+1 = 0\qquad\qquad (mod 2)$

**Remarks:**
* The final state of a cell **does not depend on the order** in which a series of toggle operations is applied!
* Applying the **same toggle operation twice leaves the state unchanged,** since $a+1+1 = a\quad (mod 2)$

<strong style="color:blue;font-size:15px;"> Encoding and Toggling the State of the Board</strong>

While the obvious representation of the state of a board of size $m \times n$ is as a matrix in $\mathbb{Z}_2^{m \times n}$,<br>
it will be convenient to encode the state as a vector by storing the cell states in column major order.

The following routines convert from matrix to vector and back, as well as a display routine for the state of the board.

In [3]:
# Conversion routines: board to vector, vector to board
to_vec(mat)        = reshape(mat, length(mat))
to_mat(vec, m, n ) = reshape(vec, m, n)

# display_routine for a board
function show_board(board)
    m,n=size(board)
    for i in 1:m
        for j in 1:n
            print(board[i,j] == 1 ? "● " : "○ ")
        end
        println()
    end
end

# capture board output
board_output( board ) = capture_output( show_board, board )
;

In [4]:
# Example
println("Example board state")
ex_board=[1 0 0; 1 1 0]
show_board(ex_board)              # prints immediately
println()
println( board_output(ex_board) ) # explicitely print captured output

Example board state
● ○ ○ 
● ● ○ 

● ○ ○ 
● ● ○ 



## 1.2 List of Possible Activations

Clicking a cell in the board activates the cell and its immediate neighbors.<br>
We now encode the activations for each of the possible cell selections.<br>

**Note that the activation patterns differ for corner, side and central cells.**

In [ ]:
function create_activation_pattern(m::Int, n::Int, row::Int, col::Int)
    pattern           = zeros(Int, m, n)
    pattern[row, col] = 1

    if row > 1  pattern[row-1, col] = 1  end  # Up
    if row < m  pattern[row+1, col] = 1  end  # Down
    if col > 1  pattern[row, col-1] = 1  end  # Left
    if col < n  pattern[row, col+1] = 1  end  # Right

    pattern
end

function create_all_patterns(m::Int, n::Int)
    patterns = dict = Dict{Tuple{Int, Int}, Array}()
    for row in 1:m
        for col in 1:n
            patterns[(row,col)] = create_activation_pattern(m, n, row, col)
        end
    end
    return patterns
end;

In [6]:
SZ = (3,4)
println( "All activation patterns for a board of size $SZ")
patterns = create_all_patterns(SZ...)
#for p in patterns show_board(p); println() end

All activation patterns for a board of size (3, 4)


Dict{Tuple{Int64, Int64}, Array} with 12 entries:
  (1, 2) => [1 1 1 0; 0 1 0 0; 0 0 0 0]
  (3, 1) => [0 0 0 0; 1 0 0 0; 1 1 0 0]
  (1, 3) => [0 1 1 1; 0 0 1 0; 0 0 0 0]
  (1, 4) => [0 0 1 1; 0 0 0 1; 0 0 0 0]
  (3, 2) => [0 0 0 0; 0 1 0 0; 1 1 1 0]
  (3, 3) => [0 0 0 0; 0 0 1 0; 0 1 1 1]
  (2, 1) => [1 0 0 0; 1 1 0 0; 1 0 0 0]
  (3, 4) => [0 0 0 0; 0 0 0 1; 0 0 1 1]
  (2, 2) => [0 1 0 0; 1 1 1 0; 0 1 0 0]
  (2, 3) => [0 0 1 0; 0 1 1 1; 0 0 1 0]
  (2, 4) => [0 0 0 1; 0 0 1 1; 0 0 0 1]
  (1, 1) => [1 1 0 0; 1 0 0 0; 0 0 0 0]

In [7]:
# Test:
show_side_by_side([ board_output(patterns[(i,j)]) for i in 1:SZ[1] for j in 1:SZ[2]])

● ● ○ ○ 
● ○ ○ ○ 
○ ○ ○ ○ 
 
 
 
 ● ● ● ○ 
○ ● ○ ○ 
○ ○ ○ ○ 
 
 
 
 ○ ● ● ● 
○ ○ ● ○ 
○ ○ ○ ○ 
 
 
 
 ○ ○ ● ● 
○ ○ ○ ● 
○ ○ ○ ○ 
 
 
 
 ● ○ ○ ○ 
● ● ○ ○ 
● ○ ○ ○ 
 
 
 
 ○ ● ○ ○ 
● ● ● ○ 
○ ● ○ ○ 
 
 
 
 ○ ○ ● ○ 
○ ● ● ● 
○ ○ ● ○ 
 
 
 
 ○ ○ ○ ● 
○ ○ ● ● 
○ ○ ○ ● 
 
 
 
 ○ ○ ○ ○ 
● ○ ○ ○ 
● ● ○ ○ 
 
 
 
 ○ ○ ○ ○ 
○ ● ○ ○ 
● ● ● ○ 
 
 
 
 ○ ○ ○ ○ 
○ ○ ● ○ 
○ ● ● ● 
 
 
 
 ○ ○ ○ ○ 
○ ○ ○ ● 
○ ○ ● ●

## 1.3 Implementing the Activation

Given a board and an activation pattern, we simply need to add the board and pattern using modulo two arithmetic

In [ ]:
activate( board, p )     = (board+p) .% 2;   # modulo 2 arithmetic
activate( board, p, m,n) = (to_mat(board,m,n) + to_mat(p,m,n)) .%2;

In [9]:
board0  = rand( 0:1, SZ...)
board1  = activate( board0, patterns[(1,1)])
board2  = activate( board1, patterns[(2,3)])
board3  = activate( board2, patterns[(2,4)])

arrow   = """<big style="font-size:50px;">&#10230;</big>"""

show_side_by_side([ board_output(board0), arrow,  board_output(board1), arrow, board_output(board2), arrow, board_output(board3)], ["Start", "", "after click 1,1", "", "after click 2,3", "", "after click 2,4"])

Start 
 ● ○ ● ● 
● ○ ● ○ 
○ ○ ○ ● 
 
 
 
 
 ⟶ 
 
 
 after click 1,1 
 ○ ● ● ● 
○ ○ ● ○ 
○ ○ ○ ● 
 
 
 
 
 ⟶ 
 
 
 after click 2,3 
 ○ ● ○ ● 
○ ● ○ ● 
○ ○ ● ● 
 
 
 
 
 ⟶ 
 
 
 after click 2,4 
 ○ ● ○ ○ 
○ ● ● ○ 
○ ○ ● ○

# 2. Puzzle Solution

## 2.1 Formulate the Solution

Given a board of size $k=M\times N$, there are $M N$ cells a player can click on, i.e., there are $M \times N$ possible activations.

Since we can commute the order of activations, and since repeating the same activation twice results in the identity,<br>
we can reach all possible board states by applying each of the activation patterns at most once, in any order.

Let $a_i,\ i=1,2\dots k$ be the activation patterns and let $b_{start}$ be the starting board state:

$\qquad$ $b_{end} = b_{start} +  \alpha_1 a_1 + \alpha_2 a_2 + \dots +\alpha_{k} a_{k}\qquad (mod 2)$

____
Given a start and end state $b_{start}, b_{end}$ we can solve for the clicks by solving for the $\alpha_i$ if possible:<br><br>
$\qquad\alpha_1 a_1 + \alpha_2 a_2 + \dots +\alpha_{k} a_{k} = b_{end}-b_{start}\qquad (mod 2)$

We can use Gaussian Elimination by converting the board states and activations into vectors, resulting in $A \alpha = b$.

**Remarks:**
* Since the entries of the vectors are in $\mathbb{Z}_2$, we have $-b_{start}=b_{start}\quad(mod 2).\quad$<br>
If $b_{end}=0,\;$ this results in the problem<br>
$\qquad A \alpha = b_{start}.$
* There is an activation vector $a_{i}$ for evey cell of a given board.<br>
The vector encode the action of a click on each of the cell entries.<br>
$\therefore A \in \mathbb{Z}_2^{K \times K},\;\;$ where $K = m n$ is the number of cells for a given board.

____
For the $3\times 4$ example given above together with a random start vector, this results in

In [ ]:
function gen_activation_matrix( m,n )
    patterns = create_all_patterns(m, n)
    stack([to_vec(patterns[(i,j)]) for i in 1:m for j in 1:n], dims=1)
end;

In [11]:
A         = gen_activation_matrix(SZ...)
board     = rand(0:1, SZ...)
b_start   = to_vec( board )
println("Initial board")
show_board(board)
println("\nSystem we need to solve:")
l_show( L"A \alpha = b_{start},\;\text{ where }\ A =", A, L"\quad b_{start} =", b_start )

Initial board
○ ● ● ● 
○ ● ● ○ 
● ● ○ ○ 

System we need to solve:


"\$A \\alpha = b_{start},\\;\\text{ where }\\ A =\$\$\\begin{equation}\n\\left(\n\\begin{array}{rrrrrrrrrrrr}\n1 & 1 & 0 & 1 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 \\\\\n1 & 0 & 0 & 1 & 1 & 0 & 1 & 0 & 0 & 0 & 0 & 0 \\\\\n0 & 0 & 0 & 1 & 0 & 0 & 1 & 1 & 0 & 1 & 0 & 0 \\\\\n0 & 0 & 0 & 0 & 0 & 0 & 1 &" ⋯ 337 bytes ⋯ " & 0 & 1 & 1 & 0 & 0 & 1 \\\\\n0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 1 & 0 & 1 & 1 \\\\\n\\end{array}\n\\right)\n\\end{equation}\n\$\$\\quad b_{start} =\$\$\\begin{equation}\n\\left(\n\\begin{array}{r}\n0 \\\\\n0 \\\\\n1 \\\\\n1 \\\\\n1 \\\\\n1 \\\\\n1 \\\\\n1 \\\\\n0 \\\\\n1 \\\\\n0 \\\\\n0 \\\\\n\\end{array}\n\\right)\n\\end{equation}\n\$"

## 2.2 Implement a GE Solver for Modulo 2 Aritmetic

In [12]:
function mod2_gaussian_elimination(A, b=:none; gj=false)
    # Check input validity
    m, n = size(A)
    aug  = b == :none ? copy(A) : [A b]
    row  = 1
    rank = 0
    pivot_cols = []
    for col in 1:n
        if (row > m) break end
        pivot_row = findfirst( aug[row:end,col] .== 1)
        if pivot_row === nothing continue end
        push!(pivot_cols, col)
        pivot_row += row-1
        rank      += 1
        if pivot_row != row  aug[row,:], aug[pivot_row,:] = aug[pivot_row,:], aug[row,:] end
        for r in row+1:m
            if aug[r,col] == 1  aug[r,:] .⊻= aug[row,:]  end # XOR for mod 2 arithmetic
        end
        if gj
        for r in 1:row-1
            if aug[r,col] == 1  aug[r,:] .⊻= aug[row,:]  end # XOR for mod 2 arithmetic
        end

        end
        row += 1;
    end

    consistent = true
    if b != :none
        for i in rank+1:m
            if aug[i,end] == 1
                consistent = false
                break
            end
        end
    end

    aug[:,1:n], aug[:,n+1:end], consistent, rank, pivot_cols  # ref(A), ref_b, consistency
end
# ---------------------------------------------------------------------------
function mod2_backsubstitution( ref_A, ref_b )
    n = size(ref_A,2)
    # Back substitution
    x = zeros(Int, n)
    for i in n:-1:1
        x[i] = ref_b[i,end]
        for j in i+1:n
            x[i] ⊻= (ref_A[i,j] & x[j])  # XOR and AND for mod 2 arithmetic
        end
    end

    return x
end;

In [13]:
# Interpret the result
number_of_steps_required( α ) = sum(α)
# ---------------------------------------------------------------------------
function click_locations_to_solve( alpha, m, n )
    indices =  [(i,j) for i in 1:m for j in 1:n]
    [v for (v, flag) in zip(indices, alpha) if flag == 1]
end;

## 2.3 Solve the Puzzle

In [14]:
function solve_board( board)
    m,n = size(board)
    A   = gen_activation_matrix(m,n)
    b   = to_vec(board)

    # solve the system
    ref_A,ref_b, consistent, rank, pivot_cols  = mod2_gaussian_elimination( A, b )
    if consistent == false
        println("inconsistent system")
        return
    end
    α = mod2_backsubstitution(ref_A, ref_b)
    A, α
end
# -------------------------------------------------------------------------------------------------
function show_solution( board, A, α )
    m,n        = size(board)
    cur_board  = copy(board)
    steps      = [ board ]
    for i in 1:size(A,2)
        if α[i] == 1
            cur_board = activate( cur_board, A[:, i], m,n)
            push!( steps, copy(cur_board) )
        end
    end

    clicks = click_locations_to_solve(α, m,n)

    boards = [ board_output(b) for b in steps]
    titles = push!(["($i,$j)" for (i,j) in clicks], "End")

    # split the displays if too long
    entries_per_sublist = 8
    l_boards = [boards[i:min(i+entries_per_sublist-1, end)] for i in 1:entries_per_sublist:length(boards)]
    l_titles = [titles[i:min(i+entries_per_sublist-1, end)] for i in 1:entries_per_sublist:length(titles)]

    for i in 1: length(l_boards)
        display(show_side_by_side(l_boards[i], l_titles[i] ))
    end
end
# -------------------------------------------------------------------------------------------------
function make_example( board )
    A, α = solve_board( board )
    println("\nNumber of steps required: $(number_of_steps_required(α))\n")
    show_solution( board, A, α )
end;

In [15]:
println("Solving a $(SZ[1])×$(SZ[2]) board")
make_example( rand(0:1, SZ[1], SZ[2] ))

Solving a 3×4 board

Number of steps required: 7



(1,1) 
 ○ ● ○ ● 
○ ○ ● ○ 
● ○ ● ○ 
 
 
 
 (1,2) 
 ● ● ○ ● 
● ● ● ○ 
● ○ ● ○ 
 
 
 
 (2,3) 
 ○ ● ○ ● 
● ○ ● ○ 
● ● ○ ○ 
 
 
 
 (2,4) 
 ○ ○ ● ● 
○ ○ ● ○ 
○ ● ○ ○ 
 
 
 
 (3,1) 
 ○ ○ ○ ● 
○ ○ ○ ● 
● ○ ○ ○ 
 
 
 
 (3,2) 
 ○ ○ ● ○ 
○ ○ ○ ○ 
● ○ ○ ● 
 
 
 
 (3,3) 
 ○ ● ● ○ 
○ ○ ● ○ 
○ ○ ○ ● 
 
 
 
 End 
 ○ ○ ○ ○ 
○ ○ ○ ○ 
○ ○ ○ ○

nothing

# 3. Solvability an Uniqueness of Solutions

## 3.1 Fundamental Subspaces of $\mathbf{\mathbb{Z}_2^{m \times n}}$

Before discussing existence and uniqueness of solutions to $A x = b$ where $A \in \mathbf{\mathbb{Z}_2^{m \times n}}$ and $b \in \mathbf{\mathbb{Z}_2^{m}}$,<br>
let us briefly review the Fundamental Theorem of Linear Algebra.

<div style="background-color:#F2F5A9;color:black;">

**Summary (Fundamental Theorem of Linear Algebra (Part I)**<br>
Let $A$ of size $M \times N$ have $rank(A) = r$ (the number of pivots in $A$),<br>
    and let Gaussian elimination result in
    $E A = R$,<br>
    where $E$ is an invertible matrix and $R$ is a row echelon form of $A$.
<div style="float:left;"><img src="Figs/FundamentalTheorem.svg" width=300/></div>
<div style="float:left;padding-left:3cm;">

| Space            | Subspace                 | Basis            | Subspace Dimension          |
|------------------|--------------------------|------------------|--------------------|
| $\mathbb{F}^N$  | $\mathscr{R}(A)$         | pivot rows of $\color{red}R$ | $rank(A)$ |
| $\mathbb{F}^N$  | $\mathscr{N}(A)$         | homogeneous solution of $A x = 0$ | $N - rank(A)$ |
| $\mathbb{F}^M$  | $\mathscr{C}(A)$         | pivot cols of $\color{red}A$ | $rank(A)$ |
| $\mathbb{F}^M$  | $\mathscr{N}(A^t)$       | homogeneous solutions of $A^t x=0$ | $M - rank(A)$ |

$$\begin{align}
\; dim\ \mathscr{C}(A) &\ + dim\ \mathscr{N}(A) &= N \\
\;\;dim\ \mathscr{R}(A) &\ + dim\ \mathscr{N}(A^t) &= M \\
\end{align}
$$
$$
dim\ \mathscr{C}(A) = dim\ \mathscr{R}(A) = rank\ A
$$
</div>
</div>

As stated, part 1 of the theorem does apply for $\mathbb{F} = \mathbb{Z}_2$<br>
$\qquad$ Part 2 of the theorem concerning orthogonality of the spaces and linear independence of orthogonal vectors **does not apply.**

### 3.1.1 Example

Consider the matrix $\; A =\begin{pmatrix} 1 & 1 \\ 1 & 1 \end{pmatrix}\; \in \mathbb{Z}_2^{2 \times 2}$

$\qquad$ The vector space $Z_2^2$ consists of exactly 4 vectors $\left\{ \begin{pmatrix} 0 \\ 0 \end{pmatrix},
\begin{pmatrix} 1 \\ 0 \end{pmatrix},\begin{pmatrix} 0 \\ 1 \end{pmatrix},\begin{pmatrix} 1 \\ 1 \end{pmatrix}\right\}$

$\qquad$ The four fundamental subspaces of $A$ are identical: they are $\;\;\left\{ \begin{pmatrix} 0 \\ 0 \end{pmatrix},
\begin{pmatrix} 1 \\ 1 \end{pmatrix}\right\}$<br>
$\qquad\qquad$ Thus, each of the fundamental subspaces has dimension 1, as required by the theorem.

**Remarks:**
* The fundamental theorem (part 1) applies.
* In general, for a matrix $A$ of size $M\times N$, the spaces $\mathscr{R}(A), \text{ and } \mathscr{N}(A)$ belong to $\mathbb{F}^N$, while $\mathscr{C}(A), \text{ and } \mathscr{N}(A^t)$ belong to $\mathbb{F}^M$. The two sets of spaces are not expected to be related in general.
* Gaussian Elimination does work, and correctly identifies bases for $\mathscr{C}(A)$ and $\mathscr{R}(A)$. Solving $A x = 0$ yields a basis for $\mathscr{N}(A)$
* For a basis for $\mathscr{N}(A^t)$ however, we need to compute the homogeneous solution of $A^t$. We cannot read it out of a Gaussian Elimination computation of $(A\ | I\ )$.
* <strong style="color:red;">FIX THE ABOVE ITEM; ALSO EXPLAIN that homogeneous solutions create click sequences that have no effect!</strong>
* We cannot combine bases to obtain a basis for the whole space.

## 3.2 Solvability and Uniqueness

An obvious question is whether a given puzzle is solvable for some board size $m \times n$.<br>
$\qquad A x = b\;\;$ is solvable iff $b \in \mathscr{C}(A),\;$ i.e., there are non-solvable system if $A$ does not have full row rank.

One way to find such systems is to complete a basis for $\mathscr{C}(A)$ to a basis for the whole space,<br>
$\qquad$ e.g., by adding a known basis and removing linearly dependent vectors from the added set.

Linear combinations of these added basis vectors will not be solvable.

In [16]:
function solvability_and_uniqueness( m, n)
    """is a puzzle of size m × n solvable for any initial board configuration? Is the solution unique?"""
    A = gen_activation_matrix(m,n)
    _, _, _, rank, _  = mod2_gaussian_elimination( A, :none )
    rank == size(A,1), size(A,2)==rank
end

println( "check various board sizes")
for m in 3:5
    for n in 3:5
        solvable, unique = solvability_and_uniqueness(m,n)
        println( "    $m × $n is always solvable: $(solvable),  \tsolutions are unique: $(unique)" )
    end
end

check various board sizes
    3 × 3 is always solvable: true,  	solutions are unique: true
    3 × 4 is always solvable: true,  	solutions are unique: true
    3 × 5 is always solvable: false,  	solutions are unique: false
    4 × 3 is always solvable: true,  	solutions are unique: true
    4 × 4 is always solvable: false,  	solutions are unique: false
    4 × 5 is always solvable: true,  	solutions are unique: true
    5 × 3 is always solvable: false,  	solutions are unique: false
    5 × 4 is always solvable: true,  	solutions are unique: true
    5 × 5 is always solvable: false,  	solutions are unique: false


In [17]:
function bases_for_solvable_and_for_unsolvable_boards( m, n)
    A = gen_activation_matrix(m,n)
    ref_A, ref_b, consistent, rank, pivot_cols  = mod2_gaussian_elimination( [A 1I] ) # the columns of I are a basis for the whole space.

    basis_for_col_A             = A[:, pivot_cols[pivot_cols .<= size(A,2)]]          # the basis for Col(A) is in A
    basis_for_unsolvable_boards = [A 1I][:, pivot_cols[pivot_cols .>  size(A,2)]]     # the additional basis vectors come from I

    [basis_for_col_A[:,i] for i in 1:size(basis_for_col_A,2)],
    [basis_for_unsolvable_boards[:,i] for i in 1:size(basis_for_unsolvable_boards,2)] # return lists of basis vectors
end

function null_space_basis( m,n )
    A = gen_activation_matrix(m,n)
    ref_A, _, _, rank, pivot_cols  = mod2_gaussian_elimination( A; gj=true )
    hs = (homogeneous_solutions( ref_A, pivot_cols ).+2).%2
    [hs[:,i] for i in 1:size(hs,2)]
end;

**Remark:** The matrices $A$ are square. If $A$ does not have full column rank, then it does not have full row rank and vice versa.<br>
$\qquad$ Hence, a given board is either uniquely solvable, or there are unsolvable boards, and no solution is unique.

<strong style="color:blue;font-size:15px;">Non Solvable Board Examples</strong>

Let's find an initial board configuration that is not solvable for a $5 \times 5$ system.

In [18]:
SZ=[5,5]
sb,ub = bases_for_solvable_and_for_unsolvable_boards(SZ...)
if length(ub) == 0
    println( "Boards of size $(SZ[1])×$(SZ[2])  are always solvable" )
else
    println("\n Basis for unsolvable configurations:\n")
    show_side_by_side( [board_output( to_mat( b, SZ...)) for b in ub] )
end


 Basis for unsolvable configurations:



● ○ ○ ○ ○ 
○ ○ ○ ○ ○ 
○ ○ ○ ○ ○ 
○ ○ ○ ○ ○ 
○ ○ ○ ○ ○ 
 
 
 
 ○ ○ ○ ○ ○ 
● ○ ○ ○ ○ 
○ ○ ○ ○ ○ 
○ ○ ○ ○ ○ 
○ ○ ○ ○ ○

In [19]:
println("\n Examples of unsolvable boards:\n")

show_side_by_side( [board_output(to_mat( (sum( sample(sb, div(length(sb),3), replace=false)) + ub[1]).%2, SZ...)) for i in 1:5])


 Example of multiple solutions:



○ ○ ● ● ○ 
● ○ ○ ○ ● 
○ ● ○ ○ ● 
● ○ ● ○ ○ 
○ ○ ○ ○ ○ 
 
 
 
 ● ○ ○ ● ○ 
○ ○ ○ ○ ○ 
● ● ● ○ ○ 
● ○ ● ○ ● 
○ ○ ○ ○ ● 
 
 
 
 ○ ○ ○ ● ● 
○ ● ○ ○ ● 
○ ○ ● ● ○ 
○ ○ ○ ○ ● 
○ ○ ○ ○ ○ 
 
 
 
 ○ ● ○ ● ● 
○ ● ○ ● ● 
● ○ ● ● ○ 
● ○ ● ○ ○ 
○ ○ ● ○ ○ 
 
 
 
 ○ ○ ○ ● ○ 
● ● ○ ● ○ 
○ ○ ● ● ● 
○ ○ ● ○ ● 
● ○ ● ● ○

In [23]:
println("check that boards with ub components are not solvable")
for k in 1:length(ub)
    for i in 1:5
       solve_board( to_mat( (sum( sample(sb, div(length(sb),3), replace=false)) + ub[k]).%2, SZ...)) # board from Col(A) + non-solvable board
    end
end

check that boards with ub components are not solvable
inconsistent system
inconsistent system
inconsistent system
inconsistent system
inconsistent system
inconsistent system
inconsistent system
inconsistent system
inconsistent system
inconsistent system


<strong style="color:blue;font-size:15px;">Multiple Solutions Example</strong>

Let's create a solvable board and obtain different solutions

In [21]:
board = to_mat( (sum( sample(sb, div(length(sb),3), replace=false))).%2, SZ...)
A,α = solve_board( board)
show_solution( board, A,α)

(2,1) 
 ● ○ ○ ○ ● 
○ ● ○ ● ○ 
○ ○ ○ ○ ○ 
○ ● ● ● ○ 
○ ○ ● ○ ○ 
 
 
 
 (2,5) 
 ○ ○ ○ ○ ● 
● ○ ○ ● ○ 
● ○ ○ ○ ○ 
○ ● ● ● ○ 
○ ○ ● ○ ○ 
 
 
 
 (3,1) 
 ○ ○ ○ ○ ○ 
● ○ ○ ○ ● 
● ○ ○ ○ ● 
○ ● ● ● ○ 
○ ○ ● ○ ○ 
 
 
 
 (3,5) 
 ○ ○ ○ ○ ○ 
○ ○ ○ ○ ● 
○ ● ○ ○ ● 
● ● ● ● ○ 
○ ○ ● ○ ○ 
 
 
 
 (4,2) 
 ○ ○ ○ ○ ○ 
○ ○ ○ ○ ○ 
○ ● ○ ● ○ 
● ● ● ● ● 
○ ○ ● ○ ○ 
 
 
 
 (4,4) 
 ○ ○ ○ ○ ○ 
○ ○ ○ ○ ○ 
○ ○ ○ ● ○ 
○ ○ ○ ● ● 
○ ● ● ○ ○ 
 
 
 
 (5,3) 
 ○ ○ ○ ○ ○ 
○ ○ ○ ○ ○ 
○ ○ ○ ○ ○ 
○ ○ ● ○ ○ 
○ ● ● ● ○ 
 
 
 
 End 
 ○ ○ ○ ○ ○ 
○ ○ ○ ○ ○ 
○ ○ ○ ○ ○ 
○ ○ ○ ○ ○ 
○ ○ ○ ○ ○

nothing

In [22]:
SZ=[5,5]
nb = null_space_basis(SZ...)
if length(nb)>0
    show_solution( board, A, α+nb[length(nb)])
end

(1,1) 
 ● ○ ○ ○ ● 
○ ● ○ ● ○ 
○ ○ ○ ○ ○ 
○ ● ● ● ○ 
○ ○ ● ○ ○ 
 
 
 
 (1,3) 
 ○ ● ○ ○ ● 
● ● ○ ● ○ 
○ ○ ○ ○ ○ 
○ ● ● ● ○ 
○ ○ ● ○ ○ 
 
 
 
 (1,5) 
 ○ ○ ● ● ● 
● ● ● ● ○ 
○ ○ ○ ○ ○ 
○ ● ● ● ○ 
○ ○ ● ○ ○ 
 
 
 
 (2,3) 
 ○ ○ ● ○ ○ 
● ● ● ● ● 
○ ○ ○ ○ ○ 
○ ● ● ● ○ 
○ ○ ● ○ ○ 
 
 
 
 (3,1) 
 ○ ○ ○ ○ ○ 
● ○ ○ ○ ● 
○ ○ ● ○ ○ 
○ ● ● ● ○ 
○ ○ ● ○ ○ 
 
 
 
 (3,5) 
 ○ ○ ○ ○ ○ 
○ ○ ○ ○ ● 
● ● ● ○ ○ 
● ● ● ● ○ 
○ ○ ● ○ ○ 
 
 
 
 (4,1) 
 ○ ○ ○ ○ ○ 
○ ○ ○ ○ ○ 
● ● ● ● ● 
● ● ● ● ● 
○ ○ ● ○ ○ 
 
 
 
 (4,2) 
 ○ ○ ○ ○ ○ 
○ ○ ○ ○ ○ 
○ ● ● ● ● 
○ ○ ● ● ● 
● ○ ● ○ ○

nothing

(4,3) 
 ○ ○ ○ ○ ○ 
○ ○ ○ ○ ○ 
○ ○ ● ● ● 
● ● ○ ● ● 
● ● ● ○ ○ 
 
 
 
 (4,4) 
 ○ ○ ○ ○ ○ 
○ ○ ○ ○ ○ 
○ ○ ○ ● ● 
● ○ ● ○ ● 
● ● ○ ○ ○ 
 
 
 
 (4,5) 
 ○ ○ ○ ○ ○ 
○ ○ ○ ○ ○ 
○ ○ ○ ○ ● 
● ○ ○ ● ○ 
● ● ○ ● ○ 
 
 
 
 (5,1) 
 ○ ○ ○ ○ ○ 
○ ○ ○ ○ ○ 
○ ○ ○ ○ ○ 
● ○ ○ ○ ● 
● ● ○ ● ● 
 
 
 
 (5,5) 
 ○ ○ ○ ○ ○ 
○ ○ ○ ○ ○ 
○ ○ ○ ○ ○ 
○ ○ ○ ○ ● 
○ ○ ○ ● ● 
 
 
 
 End 
 ○ ○ ○ ○ ○ 
○ ○ ○ ○ ○ 
○ ○ ○ ○ ○ 
○ ○ ○ ○ ○ 
○ ○ ○ ○ ○

nothing

# 4. Enhancement Ideas

Here are some ideas:
* place the board on a cylinder  (join the left and right edges of the boad)
* place the board on a Möbius strip (cross-join the top and bottom edges of the board...